In [1]:
import os, sys, glob
import time

from collections import defaultdict
from pygromos.gromos.gromosPP import GromosPP
from pygromos.gromos.gromosXX import GromosXX
from pygromos.files import imd

import restraintmaker

#CHANGE HERE
gromos_bin_path = "/home/bschroed/Documents/code/gromosPP/installed/bin"
restraintmaker_path = os.path.abspath(os.path.dirname(restraintmaker.__file__)+"/..")

control_dict = {
    "gen_resn_lib": False,
    "gen_single_tops": False, #Buggy!
    "gen_multi_state": True,
    "gen_pairwise": True,
}
sets_dir = restraintmaker_path+"/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets"
mstate_dir = sets_dir+"/multistate"
pairwise_dir = sets_dir+"/pairwise"

resn_lib_path = sets_dir+"/resn_lib.lib"

/home/bschroed/Documents/code/restraintmaker/restraintmaker/submodules/pygromos/pygromos/files/topology/top.py:20: UserWarning: Module topo-files is under Development and not entirely implemented!
  warnings.warn("Module topo-files is under Development and not entirely implemented!")


In [3]:
gromPP = GromosPP(gromos_bin_path)
atb_dirs = restraintmaker_path+"/devtools/otherScripts/b_ATB_solvationFreeEnergies/ATB_molecules"

sets_dir = restraintmaker_path+"/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets"
mstate_dir = sets_dir+"/multistate"
pairwise_dir = sets_dir+"/pairwise"

if(not os.path.exists(mstate_dir)):
    os.mkdir(mstate_dir)

if (not os.path.exists(pairwise_dir)):
    os.mkdir(pairwise_dir)

sys.path.append(atb_dirs+"/..")
import utils_test_set_ATB as util


#Systems
##get all_single file_tops:
all_tops = glob.glob(atb_dirs+"/*/*top")
state_all_tops={os.path.basename(value).split(".")[0]: value for value in all_tops}
all_mstate_sys = glob.glob(mstate_dir+"/*")

##multi_state different_sets
import utils_test_set_ATB as utils

for name,state_set in utils.multistate_ligand_sets.items():
    print(name, state_set)
    system_tops = [state_all_tops[x] for x in state_set]

    #build eds system
    out_dir = list(filter(lambda x: name+"_" in x, all_mstate_sys))[0]
    out_prefix_path = out_dir+"/"+os.path.basename(out_dir)
    out_top_path, out_ptp_path = gromPP.prep_eds(in_top_paths=system_tops, number_of_eds_states=len(system_tops), out_file_path=out_prefix_path)

    #generate cnf
    in_pdb = glob.glob(out_dir+"/*.pdb")[0]
    out_cnf_path = gromPP.pdb2gromos(in_pdb_path=in_pdb, in_top_path=out_top_path, in_lib_path=resn_lib_path,
                                     out_cnf_path=out_prefix_path+".cnf")
    print(out_cnf_path)

    #build posres/refpos
    from pygromos.files.coord import cnf
    cnf_file = cnf.Cnf(out_cnf_path)
    cnf_file.supress_atomPosition_singulrarities()
    cnf_file.write_refpos(out_prefix_path+".rfp")
    cnf_file.write_possrespec(out_prefix_path+".por", residues=list(cnf_file.residues.keys()))
    cnf_file.write(out_cnf_path)

    #generate_solvatisation
    from pygromos.data import solvents
    out_box_cnf_path=out_prefix_path+"_box.cnf"
    out_solv_cnf_path=out_prefix_path+"_spc.cnf"

    gromPP.build_box(in_top_path=out_top_path, in_cnf_path=out_cnf_path, out_cnf_path=out_box_cnf_path)

    time.sleep(2)
    box_cnf = cnf.Cnf(out_box_cnf_path)
    box_cnf.GENBOX.length = [3.610220118 for x in range(3)]
    box_cnf.write(out_box_cnf_path)
    time.sleep(2)

    gromPP.sim_box(in_top_path=out_top_path, minwall=None, in_cnf_path=out_box_cnf_path, in_solvent_cnf_file_path=solvents.spc,
                   out_cnf_path=out_solv_cnf_path, periodic_boundary_condition=None, gathering_method=None,  boxsize=True)
    time.sleep(2)

    os.remove(out_box_cnf_path)

all ['_O6T', '_O71', 'G277', 'S002', 'M097', '8018', 'M030', '6KET', 'F313', '_P8I', 'M097', 'G078', 'M218', '6J29', 'G209', 'E1VB', 'TVVS', '_O70']
/home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/all_fifthteen/all_fifthteen.cnf
/home/bschroed/Documents/code/gromosPP/installed/bin/build_box @topo /home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/all_fifthteen/all_fifthteen.top @pos /home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/all_fifthteen/all_fifthteen.cnf @nsm 1 @dens 1.0 > /home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/all_fifthteen/all_fifthteen_box.cnf 

/home/bschroed/Documents/code/gromosPP/installed/bin/sim_box @topo /home/bschroed/Documents/code/restraintmaker/devtools/otherScripts/b_ATB_solvationFreeEnergies/sets/multistate/al